In [ ]:
import pandas as pd
import os


In [ ]:
data_path = './assets/data.xlsx'
if not os.path.isfile(data_path):
  from google_drive_downloader import GoogleDriveDownloader as gdd

  data_path = './data.xlsx'
  gdd.download_file_from_google_drive(file_id='1psRf7Nyg_iZPX5fkGq19RGkr6M08jKAd',
                                      dest_path=data_path,
                                      overwrite=True)

In [ ]:
## loads excel
xls = pd.ExcelFile(data_path)
training_data = pd.read_excel(xls, '학습데이터')
priority = pd.read_excel(xls, '우선순위')
data_without_ground_truth = training_data.values[:, 2]
!pip install konlpy

In [ ]:
## Kkma sentence
from konlpy.tag import *
import copy
import time
kkma = Kkma()
okt = Okt()
kom = Komoran()


def perform_spacing_Okt(text):
  tagged = okt.pos(text)
  corrected = ""
  for i in tagged:
      if i[1] in ('Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation', 'Unknown'):
          corrected += i[0]
      else:
          corrected += " "+i[0]
  if corrected[0] == " ":
      corrected = corrected[1:]
  return corrected


def perform_Kkma(text):
  # temp_okt = perform_spacing_Okt(text)
  # temp_kkma = kkma.sentences(temp_okt)
  empt = []
  pos_kkma = kkma.pos(text)
  # print(pos_kkma)
  for i in pos_kkma:
      if i[1] == "UN":
          for j in kkma.pos(i[0]):
              # print(j)
              empt.append(j)
      else:
          empt.append(i)
  return empt


s = time.time()
training_data_copy_kkma = copy.deepcopy(training_data)
column_added_kkma = []
for text in training_data['발화']:
    # print(text)
    column_added_kkma.append(perform_Kkma(text))
training_data_copy_kkma['Kkma'] = column_added_kkma

print(training_data_copy_kkma.values[:, 10][:20])
e = time.time()
print(e - s)